Obtenga la matriz de distancias en días entre fechas de actualización de las aplicaciones pagas. ¿Cuáles son las dos aplicaciones con mayor distancia?

# Cargo el csv y hago ajustes necesarios.

In [13]:
! pip install pyspark
! pip install -U -q PyDrive
! apt update
! apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
18 packages can be upgraded. Run 'apt list --upgradable' to see them.
openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly ins

In [14]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
from pyspark.sql.types import IntegerType
from datetime import datetime
import numpy

In [15]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [16]:
downloaded = drive.CreateFile({"id" : "1-oPVltZP-8bpOqQtlRLD6yy6TNasoHLP"})
downloaded.GetContentFile("googleplaystore.csv")

In [17]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Skipeo una línea con el nombre de la aplicación respectivo porque está mal parseada.

In [18]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv("googleplaystore.csv", header = True, inferSchema = True)
rdd = df.rdd.filter(lambda x: x.App != "Life Made WI-Fi Touchscreen Photo Frame")

# Comienzo del ejercicio.

Uso take en cada celda para la revisión y sé que no es necesario.

In [19]:
rdd_paid = rdd.filter(lambda x: x.Type == "Paid")
rdd_paid.take(5)

[Row(App='TurboScan: scan documents and receipts in PDF', Category='BUSINESS', Rating=4.7, Reviews='11442', Size='6.8M', Installs='100,000+', Type='Paid', Price='$4.99', Content Rating='Everyone', Genres='Business', Last Updated='March 25, 2018', Current Ver='1.5.2', Android Ver='4.0 and up'),
 Row(App='Tiny Scanner Pro: PDF Doc Scan', Category='BUSINESS', Rating=4.8, Reviews='10295', Size='39M', Installs='100,000+', Type='Paid', Price='$4.99', Content Rating='Everyone', Genres='Business', Last Updated='April 11, 2017', Current Ver='3.4.6', Android Ver='3.0 and up'),
 Row(App='TurboScan: scan documents and receipts in PDF', Category='BUSINESS', Rating=4.7, Reviews='11442', Size='6.8M', Installs='100,000+', Type='Paid', Price='$4.99', Content Rating='Everyone', Genres='Business', Last Updated='March 25, 2018', Current Ver='1.5.2', Android Ver='4.0 and up'),
 Row(App='Tiny Scanner Pro: PDF Doc Scan', Category='BUSINESS', Rating=4.8, Reviews='10295', Size='39M', Installs='100,000+', Type=

In [20]:
rdd_paid = rdd_paid.map(lambda x: (1, (x.App, datetime.strptime(x["Last Updated"], "%B %d, %Y"))))
rdd_paid.take(5)

[(1,
  ('TurboScan: scan documents and receipts in PDF',
   datetime.datetime(2018, 3, 25, 0, 0))),
 (1, ('Tiny Scanner Pro: PDF Doc Scan', datetime.datetime(2017, 4, 11, 0, 0))),
 (1,
  ('TurboScan: scan documents and receipts in PDF',
   datetime.datetime(2018, 3, 25, 0, 0))),
 (1, ('Tiny Scanner Pro: PDF Doc Scan', datetime.datetime(2017, 4, 11, 0, 0))),
 (1, ('Puffin Browser Pro', datetime.datetime(2018, 7, 5, 0, 0)))]

In [21]:
rdd_paid = rdd_paid.join(rdd_paid)
rdd_paid.take(5)

[(1,
  (('TurboScan: scan documents and receipts in PDF',
    datetime.datetime(2018, 3, 25, 0, 0)),
   ('TurboScan: scan documents and receipts in PDF',
    datetime.datetime(2018, 3, 25, 0, 0)))),
 (1,
  (('TurboScan: scan documents and receipts in PDF',
    datetime.datetime(2018, 3, 25, 0, 0)),
   ('Tiny Scanner Pro: PDF Doc Scan', datetime.datetime(2017, 4, 11, 0, 0)))),
 (1,
  (('TurboScan: scan documents and receipts in PDF',
    datetime.datetime(2018, 3, 25, 0, 0)),
   ('TurboScan: scan documents and receipts in PDF',
    datetime.datetime(2018, 3, 25, 0, 0)))),
 (1,
  (('TurboScan: scan documents and receipts in PDF',
    datetime.datetime(2018, 3, 25, 0, 0)),
   ('Tiny Scanner Pro: PDF Doc Scan', datetime.datetime(2017, 4, 11, 0, 0)))),
 (1,
  (('TurboScan: scan documents and receipts in PDF',
    datetime.datetime(2018, 3, 25, 0, 0)),
   ('Puffin Browser Pro', datetime.datetime(2018, 7, 5, 0, 0))))]

In [22]:
rdd_cartesiano = rdd_paid.map(lambda x: ((x[1][0][0], x[1][0][1]), (x[1][1][0], x[1][1][1])))
rdd_cartesiano.take(5)

[(('TurboScan: scan documents and receipts in PDF',
   datetime.datetime(2018, 3, 25, 0, 0)),
  ('TurboScan: scan documents and receipts in PDF',
   datetime.datetime(2018, 3, 25, 0, 0))),
 (('TurboScan: scan documents and receipts in PDF',
   datetime.datetime(2018, 3, 25, 0, 0)),
  ('Tiny Scanner Pro: PDF Doc Scan', datetime.datetime(2017, 4, 11, 0, 0))),
 (('TurboScan: scan documents and receipts in PDF',
   datetime.datetime(2018, 3, 25, 0, 0)),
  ('TurboScan: scan documents and receipts in PDF',
   datetime.datetime(2018, 3, 25, 0, 0))),
 (('TurboScan: scan documents and receipts in PDF',
   datetime.datetime(2018, 3, 25, 0, 0)),
  ('Tiny Scanner Pro: PDF Doc Scan', datetime.datetime(2017, 4, 11, 0, 0))),
 (('TurboScan: scan documents and receipts in PDF',
   datetime.datetime(2018, 3, 25, 0, 0)),
  ('Puffin Browser Pro', datetime.datetime(2018, 7, 5, 0, 0)))]

La matriz de distancias en días entre fechas de actualización de las aplicaciones pagas es:

In [23]:
rdd_distancias = rdd_cartesiano.map(lambda x: ((x[0][0], x[1][0]), np.absolute((x[0][1] - x[1][1]).days)))
rdd_distancias.take(5)

[(('TurboScan: scan documents and receipts in PDF',
   'TurboScan: scan documents and receipts in PDF'),
  0),
 (('TurboScan: scan documents and receipts in PDF',
   'Tiny Scanner Pro: PDF Doc Scan'),
  348),
 (('TurboScan: scan documents and receipts in PDF',
   'TurboScan: scan documents and receipts in PDF'),
  0),
 (('TurboScan: scan documents and receipts in PDF',
   'Tiny Scanner Pro: PDF Doc Scan'),
  348),
 (('TurboScan: scan documents and receipts in PDF', 'Puffin Browser Pro'),
  102)]

Las dos aplicaciones con mayor distancia entre ellas son:

In [24]:
rdd_distancias.reduce(lambda x, y: x if x[1] > y[1] else y)

(('Fast Tract Diet', 'AH Alarm Panel'), 2645)